# ELISA - Explore the Data

## Python Environment

In [1]:
import os
import pandas as pd
import seaborn as sns
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import (Column, DateTime, Integer, Numeric, Float, String, Text)

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from dotenv import load_dotenv

from IPython.display import HTML, display

## User Environment

In [2]:
env_path = Path('.')/'.env'
load_dotenv(".env")
DR_USER = os.environ.get('DR_USER')
DR_PASSWORD = os.environ.get('DR_PASSWORD')
DR_SERVER = os.environ.get('DR_SERVER')
DR_DATABASE = os.environ.get('DR_DATABASE')
SHARED_USER = os.environ.get('SHARED_USER')
SHARED_PASSWORD = os.environ.get('SHARED_PASSWORD')
SHARED_SERVER = os.environ.get('SHARED_SERVER')
SHARED_DATABASE = os.environ.get('SHARED_DATABASE')
DATA_DIRECTORY = os.environ.get('DATA_DIRECTORY')

# Use the flag below to turn off or turn on the running of the SQL that
# interacts with MySQL. This is being used to hopefully allow the 
# execution of the notebook code, when you do not have a connection to
# MySQL.
RUN_SQL = True 
GEN_PLOTS = False

## Database Information

To explore and clean up the result information, we will extract data from the latest ImmPort DataRelease(DR) database, and then store the temporary and final results in a SharedData database. Much of this work and exploration could be done entirely in the notebook without the SharedData database, but we may use the SharedData database as a source for dynamic web content.

First step is to creat the SharedData database if it does not already exist. To create the new database login to the MySQL server then run the command below:

<div class="alert alert-block alert-info">
create database SharedData character set utf8
</div>

Next we will setup connections to the DR database and the SharedData database

In [3]:
conn1 = f"mysql+mysqlconnector://{DR_USER}:{DR_PASSWORD}@{DR_SERVER}:3306/{DR_DATABASE}"
dr_conn = create_engine(conn1, echo=False)
conn2 = f"mysql+mysqlconnector://{SHARED_USER}:{SHARED_PASSWORD}@{SHARED_SERVER}:3306/{SHARED_DATABASE}"
shared_conn = create_engine(conn2, echo=False)

# Initialize the ShareData ELIASA_Results Table

In [4]:
file_name = DATA_DIRECTORY + "/elisa_result_initial.txt"

if RUN_SQL:
    sql = f'''
CREATE TABLE {SHARED_DATABASE}.elisa_result AS
(
SELECT l1.immunology_symbol,
       e1.analyte_reported,
       e1.unit_reported,
       e1.unit_preferred,
       e1.value_reported,
       e1.value_preferred,
       e1.study_accession,
       e1.arm_accession,
       e1.subject_accession,
       e1.biosample_accession,
       e1.experiment_accession,
       e1.expsample_accession,
       e1.study_time_collected,
       e1.study_time_collected_unit,
       b1.type as biosample_type,
       b1.planned_visit_accession,
       p1.name as planned_visit_name,
       p1.min_start_day,
       p1.max_start_day
  FROM elisa_result e1
    JOIN biosample b1
      ON e1.biosample_accession = b1.biosample_accession
    JOIN planned_visit p1
      ON b1.planned_visit_accession = p1.planned_visit_accession
    LEFT OUTER JOIN lk_analyte l1
      ON e1.analyte_preferred = l1.analyte_accession
)
'''
    shared_conn.execute('DROP TABLE IF EXISTS elisa_result')
    dr_conn.execute(sql)
    shared_conn.execute('CREATE INDEX idx_elisa_study_accession on elisa_result(study_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_arm_accession on elisa_result(arm_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_biosample_accession on elisa_result(biosample_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_experiment_accession on elisa_result(experiment_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_expsample_accession on elisa_result(expsample_accession)')
    shared_conn.execute('CREATE INDEX idx_elisa_subject_accession on elisa_result(subject_accession);')
    
    sql = '''
SELECT e1.immunology_symbol,
       e1.analyte_reported,
       e1.unit_reported,
       e1.unit_preferred,
       e1.value_reported,
       e1.value_preferred,
       e1.study_accession,
       e1.arm_accession,
       e1.subject_accession,
       e1.biosample_accession,
       e1.experiment_accession,
       e1.expsample_accession,
       e1.study_time_collected,
       e1.study_time_collected_unit,
       e1.biosample_type,
       e1.planned_visit_accession,
       e1.planned_visit_name,
       e1.min_start_day,
       e1.max_start_day
  FROM elisa_result e1
'''
    
    elisa_result = pd.read_sql(sql, shared_conn)
    elisa_result.to_csv(file_name, sep='\t')

else:
    elisa_result = pd.read_csv(file_name, sep="\t")
    

# Explore Current ELISA Results Content

Review summary information on the content in specific columns in the Elisa_result table. The kinds of information we are looking for are the number of rows in the table; have the unit_preferred and value_preferred columns been populated, what are the different unit_preferred values, etc.

## Row Counts

In [5]:
if RUN_SQL:
    sql = '''
SELECT count(*)
  FROM elisa_result
'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE unit_reported is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - unit_reported: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE unit_preferred is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - unit_preferred: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE value_reported is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - value_reported: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE value_preferred is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - value_preferred: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE analyte_reported is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - analyte_reported: ",count)

    sql = '''
SELECT count(*)
  FROM elisa_result
 WHERE immunology_symbol is not null'''
    count = shared_conn.execute(sql).fetchone()[0]
    print("Total Rows - analyte_preferred: ",count)

Total Rows:  246120
Total Rows - unit_reported:  246120
Total Rows - unit_preferred:  92555
Total Rows - value_reported:  245862
Total Rows - value_preferred:  245862
Total Rows - analyte_reported:  246120
Total Rows - analyte_preferred:  200679


## Review Null Columns

In [6]:
elisa_result.isnull().sum()

immunology_symbol             45441
analyte_reported                  0
unit_reported                     0
unit_preferred               153565
value_reported                  258
value_preferred                 258
study_accession                   0
arm_accession                     0
subject_accession                 0
biosample_accession               0
experiment_accession              0
expsample_accession               0
study_time_collected              5
study_time_collected_unit         0
biosample_type                    0
planned_visit_accession           0
planned_visit_name                0
min_start_day                     0
max_start_day                     0
dtype: int64

## Review the Analyte_Preferred and Analyte_Reported Content

Below are a couple of queries, to identify the current state of the analyte and unit columns. This information may be of use in the curation process. 

In [7]:
#elisa_result.immunology_symbol.value_counts()
analyte_preferred_count = elisa_result.groupby('immunology_symbol')['immunology_symbol'].count().to_frame()
analyte_preferred_count.rename(columns={'immunology_symbol':'count'}, inplace=True)
analyte_preferred_count

,count
immunology_symbol,
AFP,259
BAFF,183
CCL22,636
CD62L,183
CSF3,640
CTACK,72
CXCL9,3046
Eotaxin,600
GMCSF,1595


## Number of Results for Each Study and Analyte

In [8]:
#symbol_by_study = elisa_result.groupby('study_accession').immunology_symbol.value_counts()
symbol_by_study = elisa_result.sort_values(['study_accession','immunology_symbol']).groupby(['study_accession','immunology_symbol']).immunology_symbol.count()
symbol_by_study = symbol_by_study.to_frame()
symbol_by_study.rename(columns={'immunology_symbol':'count'}, inplace=True)

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [10]:
pd.set_option("display.max_rows",1000)
symbol_by_study

count
study_accession immunology_symbol       
SDY113          IgA1                  60
SDY208          IFNG                   4
                IL4                    4
                IgA1                   1
                TNFA                   4
SDY21           CSF3                 216
                Eotaxin              179
                GMCSF                177
                GRO                  220
                IFNG                 180
                IL10                 194
                IL12p40              213
                IL13                 190
                IL17                 116
                IL1A                 215
                IL1B                 237
                IL2                  188
                IL3                  174
                IL4                  157
                IL5                  121
                IL6                  175
                IL9                  212
                MCP1                 227
                MIP1A                179
                MIP1B                224
                RANTES               232
                TNFA                 206
SDY224          IgA1                 141
                IgM                  141
SDY269          Eotaxin              167
                IFNA2                167
                IFNG                 167
                IL12p70              167
                IL1A                 167
                IL1B                 167
                IP-10                167
                MCP1                 167
                MIP1A                167
                RANTES               167
SDY272          IgA1                 266
                IgM                  266
SDY28           IFNA                6211
                IFNB1               5255
                IFNG                6204
                IL10                5998
                IL12p40             5342
                IL12p70             5324
                IL18                5196
                IL1B                5518
                IL2                 5201
                IL4                 5980
                IL6                 5056
                TNFA                5506
SDY283          IFNA2                774
                IFNG                 774
                IL10                 774
                IL12p40              774
                IL12p70              774
                IL1B                 716
                IL23A                602
                IL6                  737
                TNFA                 761
SDY29           IgA1                 456
                IgM                  600
SDY305          IgA1                  23
SDY36           AFP                  259
                CSF3                 226
                Eotaxin              226
                GMCSF                725
                IFNA                 521
                IFNA2               1005
                IFNG                1246
                IGF1                 179
                IGFBP3               179
                IL10                 747
                IL12p40             1190
                IL12p70             1190
                IL13                1246
                IL15                1190
                IL17                 226
                IL1A                1190
                IL1B                1190
                IL2                  747
                IL3                  226
                IL4                  747
                IL6                 2210
                IL7                  226
                IL8                 2210
                IP-10               1190
                LEP                  179
                MCP1                 226
                MIP1A                655
                MIP1B                569
                PAPPA                259
                PIGF                 179
                RANTES               226
                T

# QC and Clean Data

## Remove Rows

Remove rows where immunology_symbol and/or value_preferred is null.

In [11]:
print(elisa_result.shape)
elisa_result = elisa_result[elisa_result['value_preferred'].notnull()]
print(elisa_result.shape)
elisa_result = elisa_result[elisa_result['immunology_symbol'].notnull()]
print(elisa_result.shape)

(246120, 19)
(245862, 19)
(200679, 19)



# Review Summary Information

## Study - Analyte - Count

In [12]:
data= []
studies = sorted(elisa_result['study_accession'].unique())
for study in studies:
    study_rows = elisa_result[elisa_result['study_accession']==study]
    analytes = sorted(study_rows['immunology_symbol'].unique())
    for analyte in analytes:
        values = study_rows[study_rows['immunology_symbol']==analyte]
        data.append([study, analyte, len(values)])

display(HTML(
    '<table><tr><th>Study</th><th>Analyte</th><th>Count</th></tr><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
))


Study,Analyte,Count
SDY113,IgA1,60
SDY208,IFNG,4
SDY208,IL4,4
SDY208,IgA1,1
SDY208,TNFA,4
SDY21,CSF3,216
SDY21,Eotaxin,179
SDY21,GMCSF,177
SDY21,GRO,220
SDY21,IFNG,180


## Study - Analyte - Planned Visit - Arm Descriptive Statistics

In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:.2f}'.format

data= []
studies = sorted(elisa_result['study_accession'].unique())
for study in studies:
    study_rows = elisa_result[elisa_result['study_accession']==study]
    analytes = sorted(study_rows['immunology_symbol'].unique())
    print(study)
    #study_rows.groupby('immunology_symbol')['value_preferred'].describe()
    study_rows.groupby(['immunology_symbol','planned_visit_accession','arm_accession'])['value_preferred'].describe()
#    for analyte in analytes:
#        values = study_rows[study_rows['immunology_symbol']==analyte]
#        print(values['value_preferred'].describe())

SDY113
SDY208
SDY21
SDY224
SDY269
SDY272
SDY28
SDY283
SDY29
SDY305
SDY36
SDY396
SDY40
SDY41
SDY471
SDY473
SDY557
SDY564
SDY571
SDY58
SDY6
SDY614
SDY622
SDY648
SDY67
SDY720
SDY739
SDY8
SDY819
SDY9
SDY91
SDY95
SDY984


# Plotting

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
if GEN_PLOTS:
    data= []
    studies = sorted(elisa_result['study_accession'].unique())
    for study in studies:
        study_rows = elisa_result[elisa_result['study_accession']==study]
        analytes = sorted(study_rows['immunology_symbol'].unique())
        for analyte in analytes:
            analyte_rows = study_rows[study_rows['immunology_symbol']==analyte]
            #print(analyte_rows.boxplot('value_preferred', by='arm_accession'))
            fig, ax = plt.subplots(1,sharex=True)
            fig.set_size_inches(20,8)
            sns.boxplot(x='planned_visit_accession', y='value_preferred',
                    data=analyte_rows, hue='arm_accession', ax=ax);
            ax.set_title(study + ": " + analyte)
            ax.set_xlabel("")
            plt.show();
            plt.close(fig);